In [70]:
import pandas as pd
import numpy as np
import os
import talib
from talib import MA_Type
from os import listdir
from os.path import join, isfile


root_path = os.getcwd()
hist_day_sub_path = "data/history/day"
to_csv_path = "data/quant/day"
hist_day_path = join(root_path, hist_day_sub_path)
#print (hist_day_path)
day_stock_file_list = [ join(hist_day_path,f) for f in listdir(hist_day_path) if isfile(join(hist_day_path, f)) ]
#print (day_stock_file_list)
#print (len(day_stock_file_list)) 

def format_result(item):
    if (item['close'] > item['last_close']):
        return 1
    elif (item['close'] == item['last_close']):
        return 0
    else:
        return -1

for file in day_stock_file_list:
    df = pd.read_csv(file)
    df.round(3)
    
    ## add last_close
    df['last_close']= df.close.shift(1)
    df.last_close[0] = df.open[0]
    
    ## add upper_limit
    df['upper_limit'] = (df.last_close * 1.1)
    ## add low_limit
    df['low_limit'] = (df.last_close * 0.9)
    
    ## add close_upper_limit
    df['close_upper_limit'] = (df.close.round(2) >= (df.upper_limit - 0.001).round(2)).astype(int)
    ## add close_low_limit
    df['close_low_limit'] = (df.close.round(2) <= (df.low_limit + 0.001).round(2)).astype(int)
    
    ##Boll (20,2)
    df['boll_upper'], df['boll_middle'], df['boll_lower'] = talib.BBANDS(df.close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    df['close_boll_upper'] = (df.close > df.boll_upper).astype(int)
    df['close_boll_lower'] = (df.close < df.boll_lower).astype(int)
    df = df.fillna(0)
    
    ##ENE
    
    ## TODO add more factor
    
    ## result:
    df['result'] = df.apply(format_result, axis=1)

    ## format
    
#     print (df[df.close_boll_upper == 1])
    print (df)
    break
    code_str = file.split("/")[-1]
    _path = join(to_csv_path, code_str)
    
    df.round(3).to_csv(_path, index=False)
    break

           date    open   close    high     low    volume    code  last_close  \
0    2016-09-20   6.197   6.923   6.923   6.197      50.0  300542       6.197   
1    2016-09-21   7.614   7.614   7.614   7.614      20.0  300542       6.923   
2    2016-09-22   8.376   8.376   8.376   8.376      21.0  300542       7.614   
3    2016-09-23   9.213   9.213   9.213   9.213      32.0  300542       8.376   
4    2016-09-26  10.133  10.133  10.133  10.133      56.0  300542       9.213   
5    2016-09-27  11.146  11.146  11.146  11.146      37.0  300542      10.133   
6    2016-09-28  12.259  12.259  12.259  12.259      79.0  300542      11.146   
7    2016-09-29  13.483  13.483  13.483  13.483     122.0  300542      12.259   
8    2016-09-30  14.830  14.830  14.830  14.830      94.0  300542      13.483   
9    2016-10-10  16.312  16.312  16.312  16.312     157.0  300542      14.830   
10   2016-10-11  17.946  17.946  17.946  17.946     137.0  300542      16.312   
11   2016-10-12  19.738  19.

/home/cloud001/ly/mygithub/stock/local/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
